In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
from IPython.display import clear_output
from operator import itemgetter

import time 

In [ ]:
#Read CSV file containing the matches between NTS and SPENSER:
NTS_SPENSER_matched_dir = r'' # use your path
NTS_SPENSER_matched_file = os.path.join(NTS_SPENSER_matched_dir, "")
df_NTS_SPENSER_matched = pd.read_csv(NTS_SPENSER_matched_file, index_col=None, header=0)

In [ ]:
len(df_NTS_SPENSER_matched)

In [ ]:
NTS_id_list = df_NTS_SPENSER_matched['IndividualID'].unique().tolist()

In [ ]:
NTS_day_id_list = df_NTS_SPENSER_matched['DayID'].unique().tolist()

In [ ]:
# Import the file with the days related to each individual
trips_dir = r'' # use your path


df_NTS_trips = pd.read_csv(
    trips_dir,
    sep='\t',
    usecols=['TripID',
             'DayID',               # ID given to all trips made by an individual on a given travel day - Created in SQL
             'IndividualID',        # Individual unique ID - Created in SQL
             'HouseholdID',         # Household unique ID - Created in SQL
             'PSUID',               # PSU unique ID - Created in SQL
             'JourSeq',             # Journey number on a given travel day
             'NumStages',           # Number of stages - actual number
             'MainMode_B04ID',      # Main mode of travel - publication table breakdown - 13 categories
             'TripPurpTo_B01ID',   # Trip purpose - full list - 23 categories
             'TripStartHours',      # Trip start time - hours component
             'TripStartMinutes',    # Trip start time - minutes component
             'TripStart',           # Trip start time - minutes past midnight
             'TripEndHours',        # Trip end time - hours component
             'TripEndMinutes',      # Trip end time - minutes component
             'TripEnd',             # Trip end time - minutes past midnight
             'TripDisIncSW',        # Trip distance - including short walk - miles - actual distance
             'TripTotalTime'        # Total trip time - minutes - actual time
             ]          
)
#persons_in.head()

In [ ]:
df_NTS_trips = df_NTS_trips.loc[(df_NTS_trips['IndividualID'].isin(NTS_id_list)) & (df_NTS_trips['DayID'].isin(NTS_day_id_list))]

In [ ]:
len(df_NTS_trips)

In [ ]:
## Update transpor mode
mode_mapping = {
    1: 'walk',
     2: 'bike',
     3: 'car',  #'Car/van driver'
     4: 'car_passenger',  #'Car/van passenger'
     5: 'motorcycle',  
     6: 'car',  #'Other private transport',
     7: 'bus', #Bus in London',
     8: 'bus', #'Other local bus',
     9: 'bus', #'Non-local bus',
     10: 'metro', #'London Underground',
     11: 'train', #'Surface Rail',
     12: 'car',  #'Taxi/minicab',
     13: 'metro', #'Other public transport',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['MainMode_B04ID'] = df_NTS_trips['MainMode_B04ID'].map(mode_mapping)

In [ ]:
purp_mapping = {
    1: 'work',
     2: 'work',  #'In course of work',
     3: 'education',
     4: 'food_shop',  #'Food shopping',
     5: 'shop',  #'Non food shopping',
     6: 'medical', #'Personal business medical',
     7: 'eat',  #'Personal business eat/drink',
     8: 'other',  #'Personal business other',
     9: 'eat',  #'Eat/drink with friends',
     10: 'other',  #'Visit friends',
     11: 'leisure',  #'Other social',
     12: 'leisure',  #'Entertain/ public activity',
     13: 'leisure_sport',  #'Sport: participate',
     14: 'home',  #'Holiday: base',
     15: 'leisure_sport',  #'Day trip/just walk',
     16: 'leisure',  #'Other non-escort',
     17: 'escort_home',  #'Escort home',
     18: 'escort_work',  #'Escort work',
     19: 'escort_work',  #'Escort in course of work',
     20: 'escort_education',  #'Escort education',
     21: 'escort_leisure',  #'Escort shopping/personal business',
     22: 'escort_other',  #'Other escort',
     23: 'home',  #'Home',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['TripPurpTo_B01ID'] = df_NTS_trips['TripPurpTo_B01ID'].map(purp_mapping)

In [ ]:
# Select only those people with trip_purpose = escort_education

df_NTS_trips_escort_edu = df_NTS_trips.loc[(df_NTS_trips['TripPurpTo_B01ID'] == 'escort_education')]

In [ ]:
len(df_NTS_trips_escort_edu)

In [ ]:
# Get the IndividualID and DayID of those trips
NTS_trips_escort_edu_nts_id_list = df_NTS_trips_escort_edu['IndividualID'].unique().tolist()

NTS_trips_escort_edu_nts_day_id_list = df_NTS_trips_escort_edu['DayID'].unique().tolist()

In [ ]:
# Get the SPENSER people assigned with trip_purpose = escort_education
df_NTS_SPENSER_matched_escort_edu = df_NTS_SPENSER_matched.loc[(df_NTS_SPENSER_matched['IndividualID'].isin(NTS_trips_escort_edu_nts_id_list)) & (df_NTS_SPENSER_matched['DayID'].isin(NTS_trips_escort_edu_nts_day_id_list))]

In [ ]:
# Number of spenser agents with trip_pursose = 'escort_education'
len(df_NTS_SPENSER_matched_escort_edu)

In [ ]:
# % of population:
len(df_NTS_SPENSER_matched_escort_edu)/len(df_NTS_SPENSER_matched) * 100

In [ ]:
df_NTS_SPENSER_matched_escort_edu.head()

In [ ]:
#####################################################

In [ ]:
header_students_matched =["PID_AreaMSOA", "IndividualID", "DayID", "School_PSU", "X", "Y"]

In [ ]:
#Read CSV file containing students matched with a education_centre:
SPENSER_students_school_selected_dir = r'' # use your path
SPENSER_students_school_selected_file = os.path.join(SPENSER_students_school_selected_dir, "")
df_SPENSER_students_school_selected = pd.read_csv(SPENSER_students_school_selected_file, index_col=None, header=0, names= header_students_matched)

In [ ]:
len(df_SPENSER_students_school_selected)

In [ ]:
df_SPENSER_students_school_selected.head()

In [ ]:
## Get the HID_AreaOA column in the table df_SPENSER_students_school_selected
## Merge df_SPENSER_students_school_selected with df_NTS_SPENSER_matched to get the HID_AreaOA with the students

## Merge dataframes to combine df_NTS_SPENSER_matched_msoa with the distances travelled by each individual        
df_SPENSER_students_school_selected = pd.merge(df_SPENSER_students_school_selected, df_NTS_SPENSER_matched[['PID_AreaMSOA','HID_AreaOA']],  how='left', left_on=['PID_AreaMSOA'], right_on = ['PID_AreaMSOA'])


In [ ]:
df_SPENSER_students_school_selected

In [ ]:
#######################

In [ ]:
# Merge dataframes based on the HH_id
df_spenser_escort_edu_location = pd.merge(df_NTS_SPENSER_matched_escort_edu, df_SPENSER_students_school_selected[['HID_AreaOA','PID_AreaMSOA','School_PSU','X','Y']],  how='left', left_on=['HID_AreaOA'], right_on = ['HID_AreaOA'])


In [ ]:
# Keep only the relevant columns
df_spenser_escort_edu_location = df_spenser_escort_edu_location[['HID_AreaOA', 'PID_AreaMSOA_x', 'PID_AreaMSOA_y', 'School_PSU','X','Y']]

In [ ]:
df_spenser_escort_edu_location.rename(columns={'PID_AreaMSOA_x': 'PID_AreaMSOA', 'PID_AreaMSOA_y': 'PID_AreaMSOA_student'}, inplace=True)

In [ ]:
# Sort by column x 
## This is to keep the ones matched in the top
### then, I will remove duplictes and keep the first (just in case spenser agent trip_purpose = escort_education has been linked more than once)
df_spenser_escort_edu_location =  df_spenser_escort_edu_location.sort_values('X')

In [ ]:
df_spenser_escort_edu_location

In [ ]:
len(df_spenser_escort_edu_location)

In [ ]:
# not null rows
len(df_spenser_escort_edu_location.loc[~(df_spenser_escort_edu_location['School_PSU'].isna())])

In [ ]:
# % of not null rows of the total spenser_escort_edu_location (175211)
139516/175211 * 100

In [ ]:
# Path to save the file as csv
spenser_escort_edu_location_path = r''

In [ ]:
# Save the file as csv file
df_spenser_escort_edu_location.to_csv(spenser_escort_edu_location_path,  encoding='utf-8', mode='a', index=False, header=['HID_AreaOA','PID_AreaMSOA','PID_AreaMSOA_student','School_PSU','X','Y'])

In [ ]:
#############################################